In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
import pyspark.sql as sql

In [2]:
conf = SparkConf().setAppName("Lab1").setMaster('yarn')

In [3]:
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [4]:
!hadoop fs -ls

Found 4 items
drwxr-xr-x   - yanashurinova yanashurinova          1 2023-11-08 16:09 .sparkStaging
-rwxr-xr-x   3 yanashurinova yanashurinova       5647 2023-11-08 15:53 station.csv
-rwxr-xr-x   3 yanashurinova yanashurinova   80208848 2023-11-08 15:53 trip.csv
-rwxr-xr-x   3 yanashurinova yanashurinova     438063 2023-11-08 15:55 weather.csv


In [4]:
warandpeace = sc.textFile("warandsociety.txt")
print(warandpeace.count())

12851


In [5]:
nilFile = sc.textFile("nil")
warandpeace.take(10)

['Лев Николаевич Толстой',
 'Война и мир. Книга 1',
 '',
 'Война и мир – 1',
 '',
 ' ',
 ' http://www.lib.ru',
 '',
 'Аннотация ',
 '']

In [6]:
linesWithWar = warandpeace.filter(lambda x: "война" in x)
linesWithWar.first()

"– Еh bien, mon prince. Genes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous previens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocites de cet Antichrist (ma parole, j'y crois) – je ne vous connais plus, vous n'etes plus mon ami, vous n'etes plus мой верный раб, comme vous dites. [Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии Бонапарте. Нет, я вас предупреждаю, если вы мне не скажете, что у нас война, если вы еще позволите себе защищать все гадости, все ужасы этого Антихриста (право, я верю, что он Антихрист) – я вас больше не знаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите.] Ну, здравствуйте, здравствуйте. Je vois que je vous fais peur, [Я вижу, что я вас пугаю,] садитесь и рассказывайте."

In [7]:
def time(f):    
    import time
    t = time.process_time()
    f()
    print(f"Elapsed time: {int((time.process_time() - t)*1e9)} ns")

linesWithWar.cache()
time(lambda: linesWithWar.count())
time(lambda: linesWithWar.count())

Elapsed time: 8914308 ns
Elapsed time: 10319094 ns


In [8]:
wordCounts = linesWithWar.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)


In [11]:
wordCounts.saveAsTextFile("warandpeace_histogram2.txt")

In [12]:
temp = sc.textFile("warandpeace_histogram2.txt")
temp.take(10)

["('–', 92)",
 "('bien,', 1)",
 "('Genes', 1)",
 "('et', 4)",
 "('Lucques', 1)",
 "('ne', 4)",
 "('plus', 3)",
 "('apanages,', 1)",
 "('de', 13)",
 "('la', 6)"]

In [13]:
#!hadoop fs -cat warandpeace_histogram.txt/*


In [ ]:
ОПЕРАЦИИ С МНОЖЕСТВАМИ


In [14]:
a = sc.parallelize([1,2,3,4])
b = sc.parallelize([3,4,6,7])


In [15]:
a.union(b).distinct().collect()

[4, 1, 2, 6, 3, 7]

In [16]:
a.intersection(b).collect()

[4, 3]

In [17]:
a.subtract(b).collect()

[1, 2]

In [18]:
broadcastVar = sc.broadcast([1,2,3])
broadcastVar.value

[1, 2, 3]

In [19]:
accum = sc.accumulator(0)
sc.parallelize([1,2,3,4]).foreach(lambda x: accum.add(x))
accum.value

10

In [20]:
pair = ('a', 'b')
print(pair[0], pair[1])

a b


In [21]:
import itertools
taxi = sc.textFile("nyctaxi.csv")
taxi=taxi.mapPartitionsWithIndex(lambda idx, it:  itertools.islice(it,1,None) if (idx==0) else it  )
for t in taxi.take(5):
    print(t)


"29b3f4a30dea6688d4c289c9672cb996","1-ddfdec8050c7ef4dc694eeeda6c4625e","2013-01-11 22:03:00",+4.07033460000000E+001,-7.40144200000000E+001,"A93D1F7F8998FFB75EEF477EB6077516","68BC16A99E915E44ADA7E639B4DD5F59",2,"2013-01-11 21:48:00",+4.06760670000000E+001,-7.39810790000000E+001,1,,+4.08000000000000E+000,900,"VTS"
"2a80cfaa425dcec0861e02ae44354500","1-b72234b58a7b0018a1ec5d2ea0797e32","2013-01-11 04:28:00",+4.08190960000000E+001,-7.39467470000000E+001,"64CE1B03FDE343BB8DFB512123A525A4","60150AA39B2F654ED6F0C3AF8174A48A",1,"2013-01-11 04:07:00",+4.07280540000000E+001,-7.40020370000000E+001,1,,+8.53000000000000E+000,1260,"VTS"
"29b3f4a30dea6688d4c289c96758d87e","1-387ec30eac5abda89d2abefdf947b2c1","2013-01-11 22:02:00",+4.07277180000000E+001,-7.39942860000000E+001,"2D73B0C44F1699C67AB8AE322433BDB7","6F907BC9A85B7034C8418A24A0A75489",5,"2013-01-11 21:46:00",+4.07577480000000E+001,-7.39649810000000E+001,1,,+3.01000000000000E+000,960,"VTS"
"2a80cfaa425dcec0861e02ae446226e4","1-aa8b16d6ae44a

In [22]:
taxiParse = taxi.map(lambda line: line.split(","))
taxiMedKey = taxiParse.map(lambda row: (row[6], 1))
taxiMedCounts = taxiMedKey.reduceByKey(lambda v1, v2: v1+v2)
top10 = taxiMedCounts.map(lambda x: x[::-1]).top(10)
for x in top10:
    print(x[::-1])

('"AB44AD9A03B7CFAF3925103BDCC0AF23"', 44)
('"71CACFBADF9568AAE88A843DB511D172"', 41)
('"6483B9BFCB216EC88986EA3AB13064E7"', 41)
('"4C73459B430339981D78795300433438"', 41)
('"67E71D24AF704D814A0A825005ADA72E"', 40)
('"02E5A4136FD0A775A023A005A4EABC62"', 40)
('"9DFBCD218E7116F34C044F0680A0FB8A"', 39)
('"8DEB70907D00AA1D7FF5E2683240549B"', 39)
('"7989C2AB3F345F4AB54D3CF1E0480D67"', 39)
('"6C9F67DF658DC5636F9E7752F203F70A"', 39)


In [23]:
taxiCounts = taxi.map(lambda line: line.split(",")).map(lambda row: (row[6],1)).reduceByKey(lambda a,b: a + b)
taxiCounts.cache()
time(lambda: taxiCounts.count())
time(lambda: taxiCounts.count())

Elapsed time: 9257292 ns
Elapsed time: 8161220 ns
